In [9]:
import pandas as pd
import json
pd.options.display.max_rows = 10000
pd.options.display.max_columns = 10000
from pandas.io.json import json_normalize
import datetime

In [10]:
#create network undirected edgelist

In [45]:
#import jan 2018 - oct 2018 citi bike trip data
bike_trips = pd.read_csv('~/Desktop/Pratt/fall2018/ad_data_vis/analysis/tripdata_2018.csv')


In [80]:
#select timeframe
start_date = '2018-08-20'
end_date = '2018-10-28'

#filter trip data by desired start and end dates
trips = bike_trips[bike_trips['starttime'] >= start_date]
trips = trips[trips['starttime'] <= end_date]

In [81]:
#import stations with nta code
stations = pd.read_csv('station_nta.csv')

In [85]:
#remove station id decimals and make all station ids strings
stations['id'] = stations['id'].fillna(0).astype(int).astype(str)
trips['start station id'] = trips['start station id'].fillna(0).astype(float).astype(int).astype(str)
trips['end station id'] = trips['end station id'].fillna(0).astype(float).astype(int).astype(str)

In [86]:
#get station id and neighborhood code
stations_nta = stations[['id', 'ntacode']]
stations_nta.head()

,id,ntacode
0,143,BK09
1,157,BK09
2,216,BK09
3,314,BK09
4,322,BK09


In [87]:
#add neighborhood info to trips

#get start station neighborhood
trips_nta = pd.merge(trips, stations_nta, how='left', left_on='start station id', right_on='id')
trips_nta = trips_nta.rename(columns={'ntacode':'start_station_neighborhood'})

#get end station neighborhood
trips_nta = pd.merge(trips_nta, stations_nta, how='left', left_on='end station id', right_on='id')
trips_nta = trips_nta.rename(columns={'ntacode':'end_station_neighborhood'})

In [88]:
trips_nta = trips_nta.drop(columns=['id_x', 'id_y'])

In [89]:
trips_nta.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,start_station_neighborhood,end_station_neighborhood
0,757,2018-08-20 00:00:09.4670,2018-08-20 00:12:47.3730,386,Centre St & Worth St,40.714948,-74.002345,394,E 9 St & Avenue C,40.725213,-73.977688,30808,Subscriber,1984,1,MN24,MN28
1,358,2018-08-20 00:00:14.6380,2018-08-20 00:06:13.5760,2023,E 55 St & Lexington Ave,40.759681,-73.970314,3136,5 Ave & E 63 St,40.766368,-73.971518,21382,Subscriber,1994,1,MN19,MN40
2,427,2018-08-20 00:00:17.7370,2018-08-20 00:07:24.7650,3314,W 95 St & Broadway,40.793770,-73.971888,3357,W 106 St & Amsterdam Ave,40.800836,-73.966449,20847,Subscriber,1999,2,MN12,MN09
3,1545,2018-08-20 00:00:22.6760,2018-08-20 00:26:08.1820,281,Grand Army Plaza & Central Park S,40.764397,-73.973715,3163,Central Park West & W 68 St,40.773407,-73.977825,28424,Customer,1969,0,MN17,MN14
4,731,2018-08-20 00:00:23.3130,2018-08-20 00:12:34.7770,3121,Jackson Ave & 46 Rd,40.745248,-73.947333,3116,Huron St & Franklin St,40.732660,-73.958260,33372,Customer,1969,0,QN31,BK76


In [92]:
trips_nta_grouped = trips_nta.groupby(['start_station_neighborhood','end_station_neighborhood'])['bikeid'].count().reset_index()

In [93]:
#use only to get undirected edgelist

#get min of start and end stations as 'source'
trips_nta_grouped['source'] = trips_nta_grouped[['start_station_neighborhood', 'end_station_neighborhood']].min(axis=1).astype(str)

#get max of start and end stations as 'target'
trips_nta_grouped['target'] = trips_nta_grouped[['start_station_neighborhood', 'end_station_neighborhood']].max(axis=1).astype(str)

#group df by source and target
trips_nta_grouped = trips_nta_grouped.groupby(['source', 'target'])['bikeid'].sum().reset_index()


In [94]:
#change 'bikeid' column name to 'weight'
trips_nta_grouped = trips_nta_grouped.rename(index=str,columns={'bikeid':'weight'})

In [96]:
trips_nta_grouped.to_csv('undir_neighborhood_network_' + start_date + '-' + end_date + '.csv', index=False)